In [1]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import predict
import optuna

In [ ]:
dataset=pd.read_csv('SolarRadiationPrediction.csv',engine='python',nrows=576*15)
dataset=dataset.drop("Data",axis=1)
dataset=dataset.drop("Time",axis=1)
#dataset=dataset.drop("Ra",axis=1)
# dataset=dataset.drop("TimeSunRise",axis=1)
# dataset=dataset.drop("TimeSunSet",axis=1)
target=dataset["Radiation"]
dataset.head(5)

,UNIXTime,Radiation,Temperature,Pressure,Humidity,WindDirection(Degrees),Speed,TimeSunRise,TimeSunSet
0,8,2.58,51,30.43,103,77.27,11.25,22020,67080
1,310,2.83,51,30.43,103,153.44,9.00,22020,67080
2,1206,2.16,51,30.43,103,142.04,7.87,22020,67080
3,1505,2.21,51,30.43,103,144.12,18.00,22020,67080
4,1809,2.25,51,30.43,103,67.42,11.25,22020,67080


In [3]:
dataset=dataset.values
target=target.values
dataset[:,1]=dataset[:,8]
dataset[:,8]=target[:]

In [4]:
def create_dataset(dataset,look_back):
    dataX,dataY=[],[]
    for i in range(len(dataset)-look_back-1):
        #a=dataset第0行i~i+look_back個
        a=dataset[i:(i+look_back),0:9]
        dataX.append(a)
        dataY.append(dataset[i+look_back,8])
    return np.array(dataX),np.array(dataY)

    

In [5]:
train_size=int(len(dataset)*0.67)
test_size=len(dataset)- train_size
train,test=dataset[0:train_size,:],dataset[train_size:len(dataset),:]


In [6]:
trainX,trainY=create_dataset(train,5)
testX,testY=create_dataset(test,5)
print("train_x: ",trainX.shape)
print("train y: ",trainY.shape)
print("testX: ",testX.shape)
print(trainX[0,:,:])
print(trainY[0])

train_x:  (5782, 5, 9)
train y:  (5782,)
testX:  (2846, 5, 9)
[[8.0000e+00 6.7080e+04 5.1000e+01 3.0430e+01 1.0300e+02 7.7270e+01
  1.1250e+01 2.2020e+04 2.5800e+00]
 [3.1000e+02 6.7080e+04 5.1000e+01 3.0430e+01 1.0300e+02 1.5344e+02
  9.0000e+00 2.2020e+04 2.8300e+00]
 [1.2060e+03 6.7080e+04 5.1000e+01 3.0430e+01 1.0300e+02 1.4204e+02
  7.8700e+00 2.2020e+04 2.1600e+00]
 [1.5050e+03 6.7080e+04 5.1000e+01 3.0430e+01 1.0300e+02 1.4412e+02
  1.8000e+01 2.2020e+04 2.2100e+00]
 [1.8090e+03 6.7080e+04 5.1000e+01 3.0430e+01 1.0300e+02 6.7420e+01
  1.1250e+01 2.2020e+04 2.2500e+00]]
2.15


In [7]:
trainX=trainX.reshape(5782,45)
testX=testX.reshape(2846,45)
trainY=trainY.reshape(len(trainY),1)
testY=testY.reshape(len(testY),1)
print(trainX.shape)
print(trainX[0,:])

(5782, 45)
[8.0000e+00 6.7080e+04 5.1000e+01 3.0430e+01 1.0300e+02 7.7270e+01
 1.1250e+01 2.2020e+04 2.5800e+00 3.1000e+02 6.7080e+04 5.1000e+01
 3.0430e+01 1.0300e+02 1.5344e+02 9.0000e+00 2.2020e+04 2.8300e+00
 1.2060e+03 6.7080e+04 5.1000e+01 3.0430e+01 1.0300e+02 1.4204e+02
 7.8700e+00 2.2020e+04 2.1600e+00 1.5050e+03 6.7080e+04 5.1000e+01
 3.0430e+01 1.0300e+02 1.4412e+02 1.8000e+01 2.2020e+04 2.2100e+00
 1.8090e+03 6.7080e+04 5.1000e+01 3.0430e+01 1.0300e+02 6.7420e+01
 1.1250e+01 2.2020e+04 2.2500e+00]


In [8]:
np.random.seed(7)


dataset=dataset.astype('float32')
target=target.astype('float32')

scalar1=MinMaxScaler(feature_range=(0,1))
scalar2=MinMaxScaler(feature_range=(0,1))
scalar3=MinMaxScaler(feature_range=(0,1))
scalar4=MinMaxScaler(feature_range=(0,1))
#scalar_dim=dataset[:,1]

#target=dataset[:,1]
trainX=scalar1.fit_transform(trainX)
testX=scalar2.fit_transform(testX)
trainY=scalar3.fit_transform(trainY)
testY=scalar4.fit_transform(testY)
#target=target.reshape(len(target),1)
# target=scalar2.fit_transform(target)


In [9]:
def objective(trial):
    # 定義 RandomForestRegressor 的超參數範圍
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 10, 200),
        'max_depth': trial.suggest_int('max_depth', 3, 20),
        'max_features': trial.suggest_categorical('max_features', ['auto', 'sqrt', 'log2']),
    }

    # 創建 RandomForestRegressor 模型
    model = RandomForestRegressor(**params)

    model.fit(trainX, trainY)
    y_pred = model.predict(testX)
    # 使用交叉驗證評估模型性能
    # 使用交叉驗證評估模型性能
    score =mean_squared_error(testY,y_pred)
    return score

In [10]:
# 使用 Optuna 調整超參數
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=10)

# 取得最佳超參數
best_params = study.best_params
print("Best Hyperparameters:", best_params)

[I 2024-01-30 18:53:42,815] A new study created in memory with name: no-name-43482f78-18c0-4d0d-8dc4-2edc2d3fa1f5
C:\Users\wang8\AppData\Local\Temp\ipykernel_28816\3276934261.py:12: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(trainX, trainY)
[I 2024-01-30 18:53:43,805] Trial 0 finished with value: 0.0070392382388831205 and parameters: {'n_estimators': 79, 'max_depth': 10, 'max_features': 'log2'}. Best is trial 0 with value: 0.0070392382388831205.
C:\Users\wang8\AppData\Local\Temp\ipykernel_28816\3276934261.py:12: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(trainX, trainY)
[I 2024-01-30 18:53:44,485] Trial 1 finished with value: 0.007964639892713173 and parameters: {'n_estimators': 35, 'max_depth': 18, 'max_features': 'log2'}. Best is trial 0 wi

Best Hyperparameters: {'n_estimators': 152, 'max_depth': 4, 'max_features': 'sqrt'}
